<h1><center>Big Data Algorithms Techniques & Platforms wtf </center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Spark and DataFrames</center>
<hr style=" border:none; height:3px;">
</h2>

# A.  Analysis of the Divine Comedy

Suppose that you have a file that contains the text of the Divine Comedy, a really famous ancient poem written in Italian. You can see an excerpt below :

"Nel mezzo del cammin di nostra vita 

mi ritrovai per una selva oscura,

ché la diritta via era smarrita.

Ahi quanto a dir qual era è cosa dura, 

esta selva selvaggia e aspra e forte, 

che nel pensier rinova la paura! 

Tant'è amara che poco è più morte; 

ma per trattar del ben ch'i' vi trovai, 

dirò de l'altre cose ch'i' v'ho scorte.

Io non so ben ridir com'i' v'intrai, 

tant'era pien di sonno a quel punto

che la verace via abbandonai." 
 


You can find here one translation for giving you an idea about the content if you do not know the poem.

"Midway upon the journey of our life

I found myself within a forest dark,

For the straightforward pathway had been lost.

Ah me! how hard a thing it is to say

What was this forest savage, rough, and stern,

Which in the very thought renews the fear. 

So bitter is it, death is little more;

But of the good to treat, which there I found,

Speak will I of the other things I saw there.

I cannot well repeat how there I entered,

So full was I of slumber at the moment

In which I had abandoned the true way."


## <strong>Exercise 0.</strong> Support functions

In [ ]:
### Write here all the import function and the support function you need for processing the text

#Executing PySpark commands
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar zxvf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext.getOrCreate(conf = conf)
print("initialization successful!")

import numpy as np
import random as rn

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

#importing the text file for processing

with open('divina_commedia.txt','r') as file:
    poem = file.read().splitlines()

#We get indices of Cantos within Inferno, Purgatorio and Paradiso
indices_of_inferno_cantos = {}
indices_of_purga_cantos = {}
indices_of_paradiso_cantos = {}
index_of_end = 0
inferno = '  Inferno • Canto'
purgatorio = '  Purgatorio • Canto'
paradiso = '  Paradiso • Canto'

#looping through the poem to create dictionary of indexes.
for x in poem:
    if inferno in x:
        indices_of_inferno_cantos[x] = poem.index(x) 
    if purgatorio in x:
        indices_of_purga_cantos[x] = poem.index(x)
    if paradiso in x:
        indices_of_paradiso_cantos[x] = poem.index(x)
    if '- - - - - - - - - - - - - - - - - - - - - - -' in x:
        index_of_end = poem.index(x)

poem_b = poem[indices_of_inferno_cantos['  Inferno • Canto I'] : index_of_end]

'''Here, we divide the poem into three sections based on the three cantoi - Inferno, Purgatorio, Paradiso.'''

#For ex, we get the index by converting the dictionary into list and then taking first index of inferno till first for purgatorio for inferno section

inferno_canti = poem[list(indices_of_inferno_cantos.items())[0][1]: list(indices_of_purga_cantos.items())[0][1]]
purga_canti = poem[list(indices_of_purga_cantos.items())[0][1]:list(indices_of_paradiso_cantos.items())[0][1]]
paradiso_canti = poem[list(indices_of_paradiso_cantos.items())[0][1]:index_of_end]




In [ ]:
#For exercise 2
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar zxvf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"


import findspark
findspark.init()

#import of the SparkSession
from pyspark.sql import SparkSession

#inizialization of the Spark Session
spark = SparkSession \
    .builder \
    .appName("Lab3") \
    .getOrCreate()

## <strong>Exercise 1</strong> Word number in verses (1 point)

Write and comment the set of Spark operations that give the average number of words in a verse.

A verse in poetry is a line of text


In [7]:
rdd = sc.parallelize(poem_b)
#Here, we remove the sentences which are headings of cantos
rdd_withoutheadings = rdd.filter(lambda x: x not in indices_of_inferno_cantos)\
                         .filter(lambda x: x not in indices_of_purga_cantos)\
                         .filter(lambda x: x not in indices_of_paradiso_cantos)
#rdd.collect()

'''Steps followed :
1. splitting our list into list of lists of words
2. removing those words which are actually spaces
3. creating a pair RDD for average purpose'''

rdd_q1 = rdd_withoutheadings.map(lambda x: x.split())\
           .filter(lambda x: len(x) > 0)\
           .map(lambda x: ('Sentence',len(x))) 
           
num_of_verses = rdd_q1.count()
total_words_in_poem = rdd_q1.reduceByKey(lambda x, y: x + y).collect()[0][1]

average_num_of_words_in_verse = total_words_in_poem / num_of_verses
print('The average number of words per verse is : ', round(average_num_of_words_in_verse,3))


The average number of words per verse is :  6.833


## <strong>Exercise 2.</strong> Punctuation (1 point)

Write and comment the set of Spark operations that give the number of verses ending with a punctuation symbol.

In [9]:
# import string library function 
import string 
result = string.punctuation #this is a string with all punctuations

#first filter gets us only those sentences which are empty i.e spaces
rdd_q2 = rdd_withoutheadings.filter(lambda x: len(x) > 0)\
                            .filter(lambda x: x[-1] in result)
                            
verses_ending_in_punctuation = rdd_q2.count()

print('The number of verses ending in punctuations are :', verses_ending_in_punctuation)

The number of verses ending in punctuations are : 9494


## <strong>Exercise 3.</strong>Word number in the text (1 point)

Write and comment the set of Spark operations that return the 30 most frequent words in the provided text.

In [22]:
'''Here I interpret the question as the 30 most frequent words in the WHOLE TEXT FILE'''

import operator
rdd_text = sc.parallelize(poem)
rdd_q3 = rdd_text.filter(lambda x: len(x) > 0)\
                            .flatMap(lambda x: x.split()) \
                            .map(lambda x: x.lower())

count_of_words =  rdd_q3.countByValue() #using countByValue and converting to a dictionary to sort

sorted_count_of_words = sorted(count_of_words.items(), key=operator.itemgetter(1),reverse=True) #sorting dictionary by value

print('30 Most frequent words in the text with their frequencies are : ',sorted_count_of_words[:31])

30 Most frequent words in the text with their frequencies are :  [('e', 3994), ('che', 3544), ('la', 2343), ('a', 1892), ('di', 1877), ('non', 1422), ('per', 1365), ('in', 1151), ('si', 1042), ('’l', 958), ('le', 794), ('li', 779), ('sì', 740), ('mi', 736), ('io', 660), ('il', 653), ('più', 645), ('con', 642), ('come', 622), ('da', 619), ('è', 604), ('de', 598), ('lo', 586), ('del', 568), ('al', 505), ('ma', 494), ('tu', 441), ('se', 414), ('ne', 384), ('quel', 367), ('nel', 307)]


In [ ]:
'''Another way to do it is interpretation where we remove stopwords of both italian and english from the most frequent words'''

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
e_stopwords = stopwords.words('english')
i_stopwords = stopwords.words('italian')

i_stopwords


import operator
rdd_text = sc.parallelize(poem)
rdd_q3 = rdd_text.filter(lambda x: len(x) > 0)\
                            .flatMap(lambda x: x.split()) \
                            .map(lambda x: x.lower()) \
                            .filter(lambda x: x not in e_stopwords)\
                            .filter(lambda x: x not in i_stopwords) 
                            

count_of_words =  rdd_q3.countByValue() #using countByValue and converting to a dictionary to sort

sorted_count_of_words = sorted(count_of_words.items(), key=operator.itemgetter(1),reverse=True) #sorting dictionary by value

print('30 Most frequent words in the text with their frequencies are : ',sorted_count_of_words[:31])

## <strong>Exercise 4.</strong> Beginning word (1 point)

Write and comment the set of Spark operations that return the most frequent word at the beginning of a verse and the number of times they appear.

In [ ]:
rdd_q4 = rdd_withoutheadings.filter(lambda x: len(x) > 0)\
                            .map(lambda x: x.split()) \
                            .map(lambda x: x[0])
                          
frequency_of_first_words = rdd_q4.countByValue() #a dictionary with first words and corresponding frequencies

#printing the most frequent word at the beginning of a verse
most_freq_fword = max(frequency_of_first_words.items() , key= lambda k: k[1])
print('The most frequent first word is :' , most_freq_fword[0], 'and it appears', most_freq_fword[1], 'times.' )


The most frequent first word is : e and it appears 1056 times.


##<strong>Exercise 5.</strong>  Ending Word (1 point)

Write and comment the set of Spark operations that return the 20 most frequent word at the end of a verse and the number of times they appear.

In [ ]:
rdd_q5 = rdd_withoutheadings.filter(lambda x: len(x) > 0)\
                            .map(lambda x: x.split()) \
                            .map(lambda x: x[-1])
                          
frequency_of_last_words = rdd_q5.countByValue() #a dictionary with first words and corresponding frequencies

#printing the most frequent word at the beginning of a verse

sorted_most_freq_lword = sorted(frequency_of_last_words.items(), key=operator.itemgetter(1),reverse=True)
print('The 20 most frequent last words are :' , sorted_most_freq_lword[:21] )

The 20 most frequent last words are : [('parte', 16), ('poco,', 14), ('segno', 13), ('loco', 13), ('parte,', 12), ('mio,', 11), ('riva', 11), ('disio', 11), ('mondo,', 11), ('vede,', 11), ('quella', 11), ('fui,', 10), ('cose', 10), ('puote', 10), ('vita,', 10), ('forte,', 10), ('prima', 10), ('mente', 10), ('fue', 10), ('natura', 10), ('virtute', 10)]


##<strong>Exercise 6.</strong> Ending of consecutive verses (2 points)

Write and comment the set of Spark operations that returns the most frequent pairs of words at the end of two consecutive verses.

Tant'è amara che poco è più morte; 

ma per trattar del ben ch'i' vi trovai, 

dirò de l'altre cose ch'i' v'ho scorte.

For these 3 verses we have as pairs (morte, trovai), (trovai, scorte).

In [ ]:
#The last line removes punctuations from the word
rdd_q6 = rdd_withoutheadings.filter(lambda x: len(x) > 0) \
                            .map(lambda x: x.split()) \
                            .map(lambda x: x[-1]) \
                            .map(lambda x: x.translate(str.maketrans('','', string.punctuation)))
rdd_new = rdd_q6.zipWithIndex()
#we now create two rdds, use the index as key, join them, and then drop the index
original_rdd = rdd_new.map(lambda x: (x[1], x[0]))
shifted_rdd = rdd_new.map(lambda x: (x[1]-1, x[0]))
results = original_rdd.join(shifted_rdd)
abc = results.values()

#Now we can apply reducebyKey and get the number of repetitions
result = abc.map(lambda x: (x,1))\
          .reduceByKey(lambda x,y: x+y)\
          .sortBy(lambda f: f[1], ascending=False)

print('The top 10 most repeating pairs are :')
result.take(10)

The top 10 most repeating pairs are :


[(('puote', 'dimandare»'), 2),
 (('messo', 'segno'), 2),
 (('mio', 'fatto'), 2),
 (('quetarsi', 'move'), 2),
 (('segno', 'esso'), 2),
 (('passo', 'viva'), 2),
 (('pace', 'poco'), 2),
 (('lasso', 'messo'), 2),
 (('scocca', 'l’arco'), 2),
 (('infuso', 'fece'), 2)]

## <strong>Exercise 7.</strong> Most common  words in the poem (1 point)

Write and comment the set of Spark operations that returns the 30 most common word in the poem (a poem is a set of verses).



In [33]:
'''Without removing italian stopwords'''
rdd_q7 = rdd_withoutheadings.filter(lambda x: len(x) > 0)\
                            .flatMap(lambda x: x.split())                 

count_of_words =  rdd_q7.countByValue() #using countByValue and converting to a dictionary to sort

sorted_count_of_words = sorted(count_of_words.items(), key=operator.itemgetter(1),reverse=True) #sorting dictionary by value

print('Most frequent words in the poem with their frequencies are : ',sorted_count_of_words[:31])



Most frequent words in the poem with their frequencies are :  [('e', 3613), ('che', 3535), ('la', 2261), ('di', 1822), ('a', 1805), ('non', 1338), ('per', 1319), ('in', 1071), ('si', 1042), ('’l', 958), ('le', 777), ('li', 753), ('mi', 736), ('sì', 706), ('il', 652), ('più', 637), ('con', 623), ('è', 600), ('de', 593), ('da', 593), ('come', 565), ('del', 564), ('io', 544), ('lo', 521), ('al', 498), ('tu', 405), ('E', 376), ('se', 376), ('ne', 375), ('ma', 353), ('quel', 349)]


In [34]:
'''With removing italian stopwords'''
rdd_q7 = rdd_withoutheadings.filter(lambda x: len(x) > 0)\
                            .flatMap(lambda x: x.split()) \
                            .map(lambda x: x.lower())\
                            .filter(lambda x: x not in i_stopwords)                

count_of_words =  rdd_q7.countByValue() #using countByValue and converting to a dictionary to sort

sorted_count_of_words = sorted(count_of_words.items(), key=operator.itemgetter(1),reverse=True) #sorting dictionary by value

print('Most frequent words in the poem with their frequencies are : ',sorted_count_of_words[:31])

Most frequent words in the poem with their frequencies are :  [('’l', 958), ('sì', 740), ('de', 598), ('quel', 367), ('così', 284), ('poi', 276), ('quando', 260), ('già', 255), ('là', 251), ('son', 227), ('tanto', 223), ('ch’io', 223), ('me', 222), ('qual', 201), ('occhi', 199), ('ben', 194), ('lor', 191), ('sé', 185), ('ché', 178), ('qui', 168), ('né', 167), ('com’', 165), ('fa', 161), ('pur', 159), ('ciò', 157), ('vidi', 151), ('però', 143), ('giù', 141), ('ch’i’', 136), ('prima', 135), ('tal', 133)]


## <strong>Exercise 8.</strong> Most common words by "Canto" (1 point)

The Divina Commedia is divided in 3 Canti: Inferno, Purgatorio, Paradiso.

Write and comment the set of Spark operations that returns the most common 30 words for each "Canto".

In [55]:
cantis = [(inferno_canti,'Inferno'), (purga_canti,'Purgatorio'), (paradiso_canti,'Paradiso')]
for canto,name in cantis:
    rdd_dummy = sc.parallelize(canto)
    rdd_q8 = rdd_dummy.filter(lambda x: len(x) > 0)\
                            .flatMap(lambda x: x.split())
    count_of_words =  rdd_q8.countByValue() #using countByValue and converting to a dictionary to sort

    sorted_count_of_words = sorted(count_of_words.items(), key=operator.itemgetter(1),reverse=True) #sorting dictionary by value

    print('30 most common words in',name ,'are:',sorted_count_of_words[:31])



30 most common words in Inferno are: [('e', 1215), ('che', 1121), ('la', 751), ('a', 609), ('di', 554), ('non', 452), ('’l', 420), ('per', 415), ('in', 292), ('le', 290), ('mi', 288), ('li', 288), ('si', 285), ('con', 246), ('sì', 246), ('io', 212), ('tu', 195), ('de', 194), ('è', 190), ('il', 183), ('lo', 182), ('da', 175), ('più', 173), ('al', 171), ('del', 154), ('E', 154), ('come', 154), ('ne', 119), ('se', 119), ('un', 118), ('quel', 114)]
30 most common words in Purgatorio are: [('e', 1206), ('che', 1159), ('la', 712), ('di', 629), ('a', 604), ('non', 473), ('per', 458), ('in', 365), ('si', 329), ('le', 280), ('’l', 279), ('mi', 246), ('più', 229), ('li', 226), ('il', 218), ('da', 210), ('sì', 200), ('è', 200), ('con', 195), ('come', 189), ('io', 183), ('de', 179), ('del', 173), ('lo', 166), ('al', 163), ('ne', 131), ('se', 130), ('ma', 126), ('E', 111), ('ti', 106), ('tu', 105)]
30 most common words in Paradiso are: [('che', 1255), ('e', 1192), ('la', 798), ('di', 639), ('a', 59

# B. Tripadvisor European Restaurant 

<p align="justify">
<font size="3">
For running this series of exercises, we are going to use a dataset coming from <a href="https://www.kaggle.com/datasets/stefanoleone992/tripadvisor-european-restaurants">Kaggle</a>.

As stated in the description of the dataset:
"The TripAdvisor dataset includes 1,083,397 restaurants with attributes such as location data, average rating, number of reviews, open hours, cuisine types, awards, etc.

The dataset combines the restaurants from the main European countries".
</font>
</p>


### The dataset

<p align="justify">
<font size="3">
The dataset is in a .csv file, and among the columns, you can find:

<ul>
    <li> <code>Restaurant_link</code>	the link of the restaurant in TripAdvisor </li>
    <li><code>Restaurant_name</code>	The name of the restaurant</li>
    <li><code>Original_location</code>	The location of the restaurant</li>
    <li><code>Country</code>	The country</li>
    <li><code>...</code>	...</li>
</ul>
</p>
</font>

In [ ]:
! pip install kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!unzip tripadvisor_european_restaurants.csv.zip

Archive:  tripadvisor_european_restaurants.csv.zip
  inflating: tripadvisor_european_restaurants.csv  
  inflating: __MACOSX/._tripadvisor_european_restaurants.csv  


## <strong> Spark and Pandas.</strong>

For this set of exercises you must import data in Spark. After this first import you can pass any dataset to Pandas for the data analysis. At the end of each exercise (when the question is pertinent) you must return (reconvert) the dataframe in Spark.

Each time you do this conversion you must comment about this. Example:

<code> # creating a Spark dataframe </code>

<code> df = ... </code>

<code> # using Pandas and creating a Pandas dataframe </code>

<code> dfp = df. ... </code>

<code> # back to Spark </code>

<code> dfs =  ... </code>

## <strong> Exercise 9.</strong> First import (1 point)
Import the csv file in Spark DataFrame and show the structure of the dataframe.

In [26]:
# Write the command that creates (reads) a Spark DataFrame and stores the reference in the dfs variable

path="/content/drive/MyDrive/tripadvisor_european_restaurants.csv"
dfs=spark.read.format("csv").option("header","true").option("inferSchema","true").option("escape", "\"").load(path)

# show the DataFrame schema
dfs.count

<bound method DataFrame.count of DataFrame[restaurant_link: string, restaurant_name: string, original_location: string, country: string, region: string, province: string, city: string, address: string, latitude: string, longitude: string, claimed: string, awards: string, popularity_detailed: string, popularity_generic: string, top_tags: string, price_level: string, price_range: string, meals: string, cuisines: string, special_diets: string, features: string, vegetarian_friendly: string, vegan_options: string, gluten_free: string, original_open_hours: string, open_days_per_week: string, open_hours_per_week: string, working_shifts_per_week: double, avg_rating: double, total_reviews_count: double, default_language: string, reviews_count_in_default_language: double, excellent: string, very_good: double, average: double, poor: double, terrible: double, food: double, service: double, value: double, atmosphere: double, keywords: string]>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## <strong> Exercise 10.</strong> Data type (1 point)
Check and comment about the data type of each column. As you know a good data analysis always starts from a good understanding of your dataset.


In [ ]:
#Printing the data type of each column
dfs.printSchema()
'''We see that majority of the columns are strings while the numeric columns are doubles or floats. 
We can easily process these datatypes as our requirements going forward.'''

## <strong> Exercise 11.</strong> Null (1 point)
Show how many null values you have in each column.

In [37]:

# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count

num_of_nulls = dfs.select([count(when(col(c).isNull(), c)).alias(c) for c in dfs.columns])
print('Below dataframe shows count of null values in each column')
num_of_nulls.show()

Below dataframe shows column wise count of null values in each columns
+---------------+---------------+-----------------+-------+------+--------+------+-------+--------+---------+-------+------+-------------------+------------------+--------+-----------+-----------+------+--------+-------------+--------+-------------------+-------------+-----------+-------------------+------------------+-------------------+-----------------------+----------+-------------------+----------------+---------------------------------+---------+---------+-------+-----+--------+------+-------+------+----------+--------+
|restaurant_link|restaurant_name|original_location|country|region|province|  city|address|latitude|longitude|claimed|awards|popularity_detailed|popularity_generic|top_tags|price_level|price_range| meals|cuisines|special_diets|features|vegetarian_friendly|vegan_options|gluten_free|original_open_hours|open_days_per_week|open_hours_per_week|working_shifts_per_week|avg_rating|total_reviews_count|de

## <strong> Exercise 12.</strong>  Gourmandise (1 point)
    
<p align="justify">
<font size="3">
We want to find the City with the greatest number of restaurant.
    <ul>
     <li> the City name </li>
     <li> the number of restaurant </li>
</ul>
You can pass by Pandas but you must return a Spark dataframe
</font>
</p>

In [ ]:
#We groupby the spark dataframe by city and then order it in descending to show cities with most restaurants
dfs_citywise = dfs.groupBy('city').count()
dfs_citywise = dfs_citywise.orderBy('count', ascending=False).dropna()
dfs_citywise.show()

## <strong> Exercise 13.</strong>  Most common cuisine in the restaurants by country (2 points)
    
<p align="justify">
<font size="3">
We want to explore the most common types (5 types) of Cuisine in each country.
</font>
</p>

In [39]:
#We convert only the two country and cuisines columns to pandas as toPandas() is an expensive function
dfp_q13 = dfs.groupBy('country','cuisines').count().toPandas()
dfp_q13 = dfp_q13.fillna('Unknown') #removing nans

In [42]:
#processing the cuisines column which contains lists of cuisines per restaurants 
dfp_cuisines = dfp_q13['cuisines']
#we will now convert each row of cuisines column into a list of strings separated by a comma i.e abc
abc = []
for bunch in dfp_cuisines:
  if ',' in bunch:
    bunch = bunch.split(',')
    abc.append(bunch)
  else:
    abc.append([bunch])

dfp_q13['cuisines_list'] = abc
dfp_q13_a = dfp_q13[['country', 'cuisines_list', 'count']]
dfp_q13_a.columns = ['Country', 'Cuisine', 'Count']

In [ ]:
#We now use the explode() function to have every row have the same cuisine. We will then use groupby again!
exp = dfp_q13_a.explode('Cuisine')
exp['Cuisine'] = exp['Cuisine'].str.replace(" ", "")
ab = exp.groupby('Country', as_index=False)

#dicto is a dictionary of groupby items made countrywise

dicto = {}

for x in ab.groups:
  dicto[x]= ab.get_group(x)
  
for keys in dicto:
  if keys is not float and "-" not in keys and "." not in keys and keys != 'Unknown': #this is to exclude noisy country names
    bsdk = dicto[keys].groupby('Cuisine',as_index=False).count()
    bsdk['Cuisine'] = bsdk['Cuisine'].str.replace(' ', '') #removing whitespaces in strings
    bsdk.drop('Country', inplace=True, axis=1) #dropping the country column as its not needed
    bsdk = bsdk.sort_values('Count', ascending = False).reset_index(drop=True) #descending ordering
    bsdk_spark = spark.createDataFrame(bsdk) #convert back to spark dataframe as required
    print('In the country', keys, 'the top 5 cuisines are')
    bsdk_spark.show(5)


## <strong> Exercise 14.</strong>  Meal by country (2 points)
    
<p align="justify">
<font size="3">
We want to find for which meal there is the greatest offer in each country. 
</font>
</p>

In [44]:
#We convert only the two country and meals columns to pandas as toPandas() is an expensive function
dfp_q14 = dfs.groupBy('country','meals').count().toPandas()
dfp_q14 = dfp_q14.fillna('Unknown') #removing nans

In [45]:
#We use very similar code to previous question here
dfp_meals = dfp_q14['meals']

abcd = []

for bunch in dfp_meals:
  if ',' in bunch:
    bunch = bunch.split(',')
    abcd.append(bunch)
  else:
    abcd.append([bunch])

dfp_q14['meals'] = abcd
dfp_q14_a = dfp_q14[['country', 'meals', 'count']]
dfp_q14_a.columns = ['Country', 'Meals', 'Count']

In [ ]:
exp = dfp_q14_a.explode('Meals')
exp['Meals'] = exp['Meals'].str.replace(" ", "")
xy = exp.groupby('Country', as_index=False)

dicto_m = {}
for x in xy.groups:
  dicto_m[x]= xy.get_group(x)
  
for keys in dicto_m:
  if keys is not float and "-" not in keys and "." not in keys and keys != 'Unknown':
    bsdk = dicto_m[keys].groupby('Meals',as_index=False).count()
    bsdk['Meals'] = bsdk['Meals'].str.replace(' ', '')
    bsdk.drop('Country', inplace=True, axis=1) #drop the country column
    bsdk = bsdk.sort_values('Count', ascending = False).reset_index(drop=True)
    bsdk_spark = spark.createDataFrame(bsdk)
    print('In the country', keys, 'the top 5 meals are \n')
    bsdk_spark.show(5)


## <strong> Exercise 15.</strong>  Meal by country/city (3 points)
    
<p align="justify">
<font size="3">
The country and the city in which there are the best-rated restaurants.

We want to explore 1) the best-rated restaurant at all and the best-rated restaurant weighted somehow (and you must propose a "somehow you like"). 

For example suppose that in a city there is only one restaurant. How do you consider your best rating?
</font>
</p>

In [47]:
dfp_q15 = dfs.select('restaurant_name','country','city','avg_rating','total_reviews_count').toPandas()

In [48]:
#Creating and cleaning dataframe
dfp_q15['restaurant_name'] = dfp_q15['restaurant_name'].fillna('Unknown').replace(r"^ +| +$", r"", regex=True) #removing nans
dfp_q15['city'] = dfp_q15['city'].fillna('Unknown').replace(r"^ +| +$", r"", regex=True) #removing nans
dfp_q15['country'] = dfp_q15['country'].fillna('Unknown').replace(r"^ +| +$", r"", regex=True) #removing nans
dfp_q15['avg_rating'] = dfp_q15['avg_rating'].fillna(0)
dfp_q15['avg_rating'] = dfp_q15['avg_rating'].astype(float)
dfp_q15['total_reviews_count'] = dfp_q15['total_reviews_count'].fillna(0)
dfp_q15['total_reviews_count'] = dfp_q15['total_reviews_count'].astype(float)

In [ ]:
#to see top 5 restaurants in all countries along with their ratings
#We have weighted them by in descending order of number of reviews
xyz = dfp_q15.groupby('country', as_index=False)

dicto_c = {}
for x in xyz.groups:
  dicto_c[x]= xyz.get_group(x)


for keys in dicto_c:
  if keys is not float and "-" not in keys and "." not in keys and keys != 'Unknown':
    bsdk = dicto_c[keys]
    bsdk.drop('country', inplace=True, axis=1) #drop the country column
    bsdk = bsdk.sort_values(['avg_rating','total_reviews_count'], ascending = False).reset_index(drop=True)
    bsdk_spark = spark.createDataFrame(bsdk)
    print('In ', keys, 'the top 5 restaurants are: \n')
    bsdk_spark.show(5)

In [52]:
#Citywise Analysis of Best Restaurants
xyza = dfp_q15.groupby('city', as_index=False)

dicto_d = {}
for x in xyza.groups:
  dicto_d[x]= xyza.get_group(x)

dict_with_citywise_rest = {}

for keys in dicto_d:
  if keys is not float and "-" not in keys and "." not in keys and keys != 'Unknown':
    bsdk = dicto_d[keys]
    bsdk.drop(['city','country'], inplace=True, axis=1) #drop the country and city column
    bsdk = bsdk.sort_values(['avg_rating','total_reviews_count'], ascending = False).reset_index(drop=True)
    dict_with_citywise_rest[keys] = bsdk.head(5)


In [54]:
#Function to see the best restaurants in a city 
def best_5_rest_in_city(city_name):
  if city_name in dicto_d:
    print(city_name)
    mdrchd = spark.createDataFrame(dict_with_citywise_rest[city_name].head(5))
    return mdrchd.show(5)
  else:
    print('Please enter city name as it appears in the dataset')
best_rest = best_5_rest_in_city('Palaiseau')


Palaiseau
+--------------------+----------+-------------------+
|     restaurant_name|avg_rating|total_reviews_count|
+--------------------+----------+-------------------+
|        In Vino Vivo|       5.0|               18.0|
|         La Fee Cafe|       5.0|               16.0|
|     La Table du 107|       5.0|               14.0|
|Le Chichrane - Ch...|       5.0|                1.0|
|     Cafe De La Gare|       5.0|                1.0|
+--------------------+----------+-------------------+



## <strong> Bonus.</strong>  Propose your own analysis



    
<p align="justify">
<font size="3">
Propose here an analysis on the dataframe.
</font>
</p>